# Introduction

***In progress.***

This notebook looks at building a simple RPG inventory system with Python. We can imagine this inventory system as an in-game backpack, suitcase, or other similar object that contains up to X number of items, based on how many "slots" (spaces which an item can fill) the player has available. Any item takes up X number of slots.

## Imports

In [1]:
import numpy as np

# `Inventory` Class

In [2]:
class Inventory:
    
    def __init__(self, total_slots = 16, items = []):
        self.total_slots = total_slots
        self.items = items
        self.used_slots = sum([x.slots for x in self.items])
        self.open_slots = self.total_slots - self.used_slots
        
    def __len__(self):
        return len(self.items)
        
    def update_slots(self):
        self.used_slots = sum([x.slots for x in self.items])
        self.open_slots = self.total_slots - self.used_slots 
        
    def add_item(self, item):
        if item.slots < self.open_slots: 
            self.items.append(item)
            # self.items.sort() # Can't sort Items, figure out how
            self.update_slots()
        else:
            print("Inventory Full")
            
    def remove_item(self, item):
        if item in self.items:
            self.items.remove(item)
        else:
            print("Item Not In Inventory")

# `Item` Class

***Needs description.***

In [24]:
class Item:
    
    def __init__(self, slots = 1):
        self.slots = slots

# Basic Examples

Example of adding items and summing inventory:

In [4]:
# Two items exist but have not been added to our Inventory

a = Item(2)
b = Item(3)

# See, no used slots

inv = Inventory()
inv.used_slots

0

In [5]:
# We have sixteen slots to fill

inv.open_slots

16

In [6]:
# We add two Items that take up five slots total and see that reflected

inv.add_item(a)
inv.add_item(b)

inv.used_slots

5

In [7]:
# Let's try and overload our Inventory with a big Item

c = Item(20)

inv.add_item(c)

Inventory Full


In [8]:
# Voila! We stopped it from overloading and stayed at the same slots

inv.used_slots

5

# `Item` Subclasses

***Needs description.***

## `Potion`

In [9]:
class Potion(Item):
    def __init__(self, slots = 1, heal = False, effect = False):
        super().__init__(slots)
        self.heal = heal
        self.effect = effect
    
    # Add functionality that destroys potion after one use

In [10]:
x = Potion(1, 7, "Speed Up")

In [11]:
x.slots

1

In [12]:
x.effect

'Speed Up'

## `Weapon` And Subclasses

***Needs description.***

In [25]:
class Weapon(Item):
    def __init__(self, slots = 2, damage = 1, element = "normal",
                 special = False):
        super().__init__(slots)
        self.damage = damage
        self.element = element
        self.special = special
        
    def do_damage(self, target):
        if self.element in target.weak:
            target.hp_current -= (self.damage * 2)
        elif self.element in target.resist:
            target.hp_current -= (self.damage // 2)
        else:
            target.hp_current -= self.damage

In [32]:
class Blaster(Weapon):
    def __init__(self, slots = 2, damage = 1, element = "normal",
                 special = False, range_ = 75, mag = 8, ammo = "steel",
                 accuracy = 0.6):
        super().__init__(slots, damage, element, special)
        self.range_ = range_
        self.mag = mag
        self.mag_status = mag
        self.ammo = ammo
        self.accuracy = accuracy
        
    def shoot(self, target, distance):
        # Check if have ammo
        if self.mag_status > 0:
            # Check if target in range
            if distance < self.range_:
                # "Roll" to hit
                if np.random.random() < self.accuracy:
                    # Deal damage
                    self.do_damage(target)
                    # Decrement ammo
                    self.mag_status -= 1
                    print("Successful Hit")
                else:
                    self.mag_status -= 1
                    print("You Shot And Missed")
            else:
                print("Target Out Of Range")
        else:
            print("Out Of Ammo")
            
    def reload(self, ammo_source):
    # Ammo source should be Item in Inventory
        pass

# `Creature` Class

In [33]:
class Creature():
    def __init__(self, hp, resist = [], weak = []):
        self.hp_max = hp
        self.hp_current = hp
        self.resist = resist
        self.weak = weak

Example of `Creature` and `Blaster` class interacting.

In [34]:
flame_pistol = Blaster(element = 'fire', range_ = 6)
zombie = Creature(hp = 5, resist = ["poison"], weak = ["fire"])

In [35]:
flame_pistol.shoot(zombie, 3)

Successful Hit


In [36]:
# Notice the hp_current went down by 2, not 1, since it is weak to fire
zombie.hp_current

3

In [37]:
flame_pistol.mag_status

7

In [38]:
isinstance(flame_pistol, Blaster)

True